In [5]:
import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

monuseg_labels = os.listdir('./Annotations')
monuseg_labels.sort()
labels = np.zeros((0,1000, 1000, 2), dtype=np.int32)


for name in monuseg_labels:
    tree = ET.parse('./Annotations/' + name)
    root = tree.getroot()
    regions = root.find('Annotation').find('Regions')
    mask = np.zeros((1000, 1000), dtype=np.int32)
    instance_mask = np.zeros((1000, 1000), dtype=np.int32)

    idx =1
    for region in regions.findall('Region'):
        arr = np.array([],dtype=np.int32)
        vertices = region.find('Vertices')
        for vertice in vertices.findall('Vertex'):
            x = round(float(vertice.get('X')))
            y = round(float(vertice.get('Y')))
            arr = np.append(arr, [x, y])
            arr = arr.reshape(-1, 2)
        
        ins = np.zeros((1000, 1000), dtype=np.int32)
        
        cv2.fillPoly(mask, [arr], 1)
        cv2.fillPoly(ins, [arr], 1)
        ins = ins * idx
        ins[np.logical_and(instance_mask!=0,ins!=0)] = 0
        instance_mask += ins
        idx += 1
    label = np.stack([ instance_mask, mask], axis=-1)
    labels = np.concatenate([labels, label[np.newaxis, ...]], axis=0)


np.save('/root/autodl-tmp/data/monuseg/MoNuSeg2018TrainingData/labels.npy',labels)

In [6]:
import torch
import os
import numpy as np
import PIL.Image as Image

imgs = torch.zeros((0, 3, 1000, 1000), dtype=torch.float32)
img_names = os.listdir('./Tissue Images')
img_names.sort()
for name in img_names:
    img = Image.open('./Tissue Images/' + name)
    img = np.array(img)
    img = torch.from_numpy(img).float()/255
    img = img.permute(2, 0, 1)
    img = img.unsqueeze(0)
    imgs = torch.cat([imgs, img], dim=0)
torch.save(imgs,'/root/autodl-tmp/data/monuseg/MoNuSeg2018TrainingData/imgs.pt')


In [7]:
import numpy as np

data = np.load('./labels.npy')
print(data[0,...,0].max())
print(data[0,...,1].max())

480
1
